In [127]:
from tqdm import tqdm
import transformers as ppb
import json
from transformers import AutoTokenizer, AutoModel

# bert-base-cased-conversational

In [134]:
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/bert-base-cased-conversational")
model = AutoModel.from_pretrained("DeepPavlov/bert-base-cased-conversational")

Some weights of the model checkpoint at DeepPavlov/bert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Dialoghi interi

In [135]:
lst_dict = []
with open('/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/data/EMNLP_dataset/dialogues_text.txt', 'r') as f:
    lines = f.readlines()
    for l in tqdm(lines):
        tokenized = tokenizer.encode(l.replace("__eou__", "[SEP]"))[:-1]
        if len(tokenized) < 513:
            lst_dict.append({'tokens': tokenizer.convert_ids_to_tokens(tokenized), 
                             'words': ["[CLS]"] + l.replace("__eou__", "[SEP]").replace("'", " ' ").split(), 
                             'text': l.replace("__eou__", "").replace("\n", "")})

100%|██████████| 13118/13118 [00:05<00:00, 2288.96it/s]


Coppie di utterance nel dialogo.\
Es:
- 1>1 ; 1>2 ; 1>3
- 2>2 ; 2>3
- 3>3


In [122]:
lst_dict = []
with open('/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/data/EMNLP_dataset/dialogues_text.txt', 'r') as f:
    lines = f.readlines()
    for l in tqdm(lines):
        utt_lst = l.split("__eou__")
        for i, utt1 in enumerate(utt_lst):
            for utt2 in utt_lst[i:]:
                tokenized = tokenizer.encode(utt1+" [SEP] "+utt2)
                utt1 = utt1.replace("'", " ' ").replace("\n", "")
                utt2 = utt2.replace("'", " ' ").replace("\n", "")
                lst_dict.append({'tokens': tokenizer.convert_ids_to_tokens(tokenized), 
                                    'words': ["[CLS]"] + utt1.split() + ["[SEP]"] + utt2.split() + ["[SEP]"], 
                                    'text': utt1 + " " + utt2})

100%|██████████| 13118/13118 [04:54<00:00, 44.48it/s] 


In [136]:
str_to_save = json.dumps(lst_dict[:50])
with open('/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/preprocessed_conversational.json', 'w') as f:
       f.write(str_to_save)

In [140]:
!!  python /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/extract_attention.py --preprocessed-data-file /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/preprocessed_conversational.json --bert-dir /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/bert/bert-base-cased-conversational

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['Creating examples...',
 'WARNING:tensorflow:From /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/venv/lib/python3.9/site-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.',
 'Instructions for updating:',
 'non-resource variables are not supported in the long term',
 'Building BERT model...',
 '/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/bert/modeling.py:692: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.',
 '  query_layer = tfv1.layers.dense(',
 '/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/venv/lib/python3.9/site-packages/keras/legacy_tf_layers/core.py:261: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.',
 '  return layer.apply(inputs)',
 '/Users/lizzy/Des

In [142]:
!! python head_distances.py --attn-data-file /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/preprocessed_conversational_attn.pkl --outfile /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/head_distances_conversational.pkl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['Loading attention data',
 'WARNING:tensorflow:From /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/venv/lib/python3.9/site-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.',
 'Instructions for updating:',
 'non-resource variables are not supported in the long term',
 'Computing head distances',
 '1/40 = 2.5%, ELAPSED: 0.0s, ETA: 0.0s',
 '2/40 = 5.0%, ELAPSED: 0.6s, ETA: 11.0s',
 '3/40 = 7.5%, ELAPSED: 13.1s, ETA: 161.8s',
 '4/40 = 10.0%, ELAPSED: 18.6s, ETA: 167.6s',
 '5/40 = 12.5%, ELAPSED: 23.6s, ETA: 165.5s',
 '6/40 = 15.0%, ELAPSED: 39.0s, ETA: 221.2s',
 '7/40 = 17.5%, ELAPSED: 39.6s, ETA: 186.8s',
 '8/40 = 20.0%, ELAPSED: 46.4s, ETA: 185.7s',
 '9/40 = 22.5%, ELAPSED: 48.9s, ETA: 168.4s',
 '10/40 = 25.0%, ELAPSED: 62.2s, ETA: 186.6s',
 '11/40 = 27.5%, ELAPSED: 64.8s, ETA: 171.0s',
 '12/40 = 30.0%, ELAPSED: 84.3s, ETA: 196.7s',
 '13/40 = 32.5

# bert-base-uncased

In [ ]:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a 

Dialoghi interi

In [ ]:
lst_dict = []
with open('/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/data/EMNLP_dataset/dialogues_text.txt', 'r') as f:
    lines = f.readlines()
    for l in tqdm(lines):
        tokenized = tokenizer.encode(l.replace("__eou__", "[SEP]"))[:-1]
        if len(tokenized) < 513:
            lst_dict.append({'tokens': tokenizer.convert_ids_to_tokens(tokenized), 
                             'words': ["[CLS]"] + l.replace("__eou__", "[SEP]").replace("'", " ' ").split(), 
                             'text': l.replace("__eou__", "").replace("\n", "")})

100%|██████████| 13118/13118 [00:05<00:00, 2288.96it/s]


Coppie di utterance nel dialogo

In [ ]:
lst_dict = []
with open('/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/data/EMNLP_dataset/dialogues_text.txt', 'r') as f:
    lines = f.readlines()
    for l in tqdm(lines):
        utt_lst = l.split("__eou__")
        for i, utt1 in enumerate(utt_lst):
            for utt2 in utt_lst[i:]:
                tokenized = tokenizer.encode(utt1+" [SEP] "+utt2)
                utt1 = utt1.replace("'", " ' ").replace("\n", "")
                utt2 = utt2.replace("'", " ' ").replace("\n", "")
                lst_dict.append({'tokens': tokenizer.convert_ids_to_tokens(tokenized), 
                                    'words': ["[CLS]"] + utt1.split() + ["[SEP]"] + utt2.split() + ["[SEP]"], 
                                    'text': utt1 + " " + utt2})

100%|██████████| 13118/13118 [04:54<00:00, 44.48it/s] 


In [ ]:
str_to_save = json.dumps(lst_dict[:50])
with open('/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/preprocessed.json', 'w') as f:
       f.write(str_to_save)

In [133]:
!!  python /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/extract_attention.py --preprocessed-data-file /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/preprocessed.json --bert-dir /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/bert/bert-base-cased-conversational

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


['Creating examples...',
 'WARNING:tensorflow:From /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/venv/lib/python3.9/site-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.',
 'Instructions for updating:',
 'non-resource variables are not supported in the long term',
 'Traceback (most recent call last):',
 '  File "/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/extract_attention.py", line 146, in <module>',
 '    main()',
 '  File "/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/extract_attention.py", line 109, in main',
 '    example = Example(features, tokenizer, args.max_sequence_length)',
 '  File "/Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/extract_attention.py", line 29, in __init__',
 '    self.input_ids = tokenizer.convert_tokens_to_ids(self.tokens)',
 '  File "/Users/lizzy/Desktop/Universita/tesi/gi

In [126]:
!! python head_distances.py --attn-data-file /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/preprocessed_attn.pkl --outfile /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/head_distances.pkl

['Loading attention data',
 'WARNING:tensorflow:From /Users/lizzy/Desktop/Universita/tesi/git/dialogue_coherence/venv/lib/python3.9/site-packages/tensorflow/python/compat/v2_compat.py:107: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.',
 'Instructions for updating:',
 'non-resource variables are not supported in the long term',
 'Computing head distances',
 '1/49 = 2.0%, ELAPSED: 0.0s, ETA: 0.0s',
 '2/49 = 4.1%, ELAPSED: 0.4s, ETA: 8.9s',
 '3/49 = 6.1%, ELAPSED: 1.0s, ETA: 14.6s',
 '4/49 = 8.2%, ELAPSED: 1.1s, ETA: 12.3s',
 '5/49 = 10.2%, ELAPSED: 1.9s, ETA: 16.9s',
 '6/49 = 12.2%, ELAPSED: 2.2s, ETA: 15.7s',
 '7/49 = 14.3%, ELAPSED: 2.2s, ETA: 13.3s',
 '8/49 = 16.3%, ELAPSED: 3.6s, ETA: 18.7s',
 '9/49 = 18.4%, ELAPSED: 5.3s, ETA: 23.7s',
 '10/49 = 20.4%, ELAPSED: 7.3s, ETA: 28.5s',
 '11/49 = 22.4%, ELAPSED: 9.4s, ETA: 32.4s',
 '12/49 = 24.5%, ELAPSED: 10.3s, ETA: 31.7s',
 '13/49 = 26.5%, ELAPSED: 10.7s, ET